In [4]:
import torch
from torch import nn
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
def Binarize(tensor,quant_mode='det'):
    if quant_mode=='det':
        return tensor.sign()
    else:
        return tensor.add_(1).div_(2).add_(torch.rand(tensor.size()).add(-0.5)).clamp_(0,1).round().mul_(2).add_(-1)

class BinarizeLinear(nn.Linear):

    def __init__(self, *kargs, **kwargs):
        try:
            self.exempt = kwargs["exempt"]
            del kwargs["exempt"]
        except:
            self.exempt=False
        self.exempt=False
        super(BinarizeLinear, self).__init__(*kargs, **kwargs)
        

    def forward(self, input):
        if self.exempt:
#         if input.size(1) != self.excempt:
            input.data=Binarize(input.data)
        if not hasattr(self.weight,'org'):
            self.weight.org=self.weight.data.clone()
        self.weight.data=Binarize(self.weight.org)
#         print(self.weight.shape)
        out = nn.functional.linear(input, self.weight)
        if not self.bias is None:
            self.bias.org=self.bias.data.clone()
            out += self.bias.view(1, -1).expand_as(out)

        return out

In [112]:
class BinarizeConv2d(nn.Conv2d):

    def __init__(self, *kargs, **kwargs):
        try:
            self.exempt = kwargs["exempt"]
            del kwargs["exempt"]
        except:
            self.exempt=False
        self.exempt=False
        super(BinarizeConv2d, self).__init__(*kargs, **kwargs)
        

    def forward(self, input):
        if self.exempt:
#         if input.size(1) != self.excempt:
            input.data=Binarize(input.data)
        if not hasattr(self.weight,'org'):
            self.weight.org=self.weight.data.clone()
            self.bias.org=self.bias.data.clone()
        self.weight.data=Binarize(self.weight.org)
        self.bias.data=Binarize(self.bias.org)
#         print(self.weight.shape)
#         out = nn.functional.linear(input, self.weight)
        out = nn.functional.conv2d(input, self.weight, self.bias, self.stride,
                        self.padding, self.dilation, self.groups)
#         if not self.bias is None:
#             self.bias.org=self.bias.data.clone()
#             out += self.bias.view(1, -1).expand_as(out)

        return out

In [113]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = BinarizeConv2d(1, 10, kernel_size=5)
        self.conv2 = BinarizeConv2d(10, 20, kernel_size=5)
        self.conv3 = BinarizeConv2d(20, 30, kernel_size=5)
#         self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(480, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.hardtanh(F.max_pool2d(self.conv1(x), 2))
#         x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = F.hardtanh(F.max_pool2d(self.conv2(x), 1))
        x = F.hardtanh(F.max_pool2d(self.conv3(x), 1))
        x = x.view(-1, 480)
        x = F.hardtanh(self.fc1(x))
#         x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [114]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args["log_interval"] == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))



In [115]:
args = {
    "batch_size": 512,
    "test_batch_size": 4096,
    "epochs": 1,
    "lr":0.01,
    "momentum": 0.5,
    "no_cuda": False,
    "seed": 1,
    "log_interval": 10
}
    # Training settings

use_cuda = not args["no_cuda"] and torch.cuda.is_available()

torch.manual_seed(args["seed"])

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args["batch_size"], shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args["test_batch_size"], shuffle=True, **kwargs)


model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args["lr"], momentum=args["momentum"])

for epoch in range(1, args["epochs"] + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.399937
Train Epoch: 1 [5120/60000 (8%)]	Loss: 2.212451
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.030643
Train Epoch: 1 [15360/60000 (25%)]	Loss: 1.857377
Train Epoch: 1 [20480/60000 (34%)]	Loss: 1.753018
Train Epoch: 1 [25600/60000 (42%)]	Loss: 1.610895
Train Epoch: 1 [30720/60000 (51%)]	Loss: 1.517119
Train Epoch: 1 [35840/60000 (59%)]	Loss: 1.462044
Train Epoch: 1 [40960/60000 (68%)]	Loss: 1.384799
Train Epoch: 1 [46080/60000 (76%)]	Loss: 1.259014
Train Epoch: 1 [51200/60000 (85%)]	Loss: 1.246261
Train Epoch: 1 [56320/60000 (93%)]	Loss: 1.255607

Test set: Average loss: 1.1244, Accuracy: 7349/10000 (73%)



In [117]:
model.conv1.bias

Parameter containing:
tensor([ 1., -1.,  1.,  1.,  1., -1.,  1.,  1., -1., -1.], requires_grad=True)

In [111]:
print(model)

Net(
  (conv1): BinarizeConv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): BinarizeConv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv3): BinarizeConv2d(20, 30, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=480, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)


In [101]:
model = Net().to(device)
x = model.conv1(torch.randn(1,1,28,28))
# print(x.shape)
# x = model.conv2(x)
# print(x.shape)
# # print(x.view(-1, 320).shape)
# x = model.conv3(x)
# print(x.shape)
# F.max_pool2d(x, 1).shape
x=torch.randn(1,1,28,28)
x = F.hardtanh(F.max_pool2d(model.conv1(x), 2))
print(x.shape)
x = F.hardtanh(F.max_pool2d(model.conv2(x), 1))
print(x.shape)
x = F.hardtanh(F.max_pool2d(model.conv3(x), 1))
print(x.shape)

torch.Size([1, 10, 12, 12])
torch.Size([1, 20, 8, 8])
torch.Size([1, 30, 4, 4])


In [74]:
binConv.weight.shape

torch.Size([10, 1, 5, 5])

In [63]:
for data, _ in train_loader:
    print(data.shape)
    break

torch.Size([512, 1, 28, 28])


 - When both conv and FC layers are binary learning is not happening. Maybe the model is not rich enough??
 - Conv layers are computaionally expensive, so let us replace on;ly conv layers with binary conv layers.
 - When all conv layers are replaced by binary conv layers with one epoch drop from 79% to 75% accuracy
 
 normal conv:
Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(20, 30, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=480, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

binary comv:
Net(
  (conv1): BinarizeConv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): BinarizeConv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv3): BinarizeConv2d(20, 30, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=480, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)
